In [ ]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [ ]:
#scraping brand, categories, seller_info,average_rating,  review_count is available in serp

In [ ]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [ ]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)


In [ ]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

In [ ]:
job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

In [ ]:
query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

In [ ]:
serp_details

In [ ]:
df=serp_details.copy()

In [ ]:
df.drop_duplicates(subset='sku', inplace=True)
df.dropna(subset='sku', inplace=True)
df.reset_index(drop=True, inplace=True)
df['categories']=''
df['average_rating']=''
df['seller_info']=''
df['brand']=''
df

In [ ]:
for i in tqdm(range(0,20)):
    sku=df['sku'][i]
    url="https://www.ebay.com/itm/{}".format(sku)
    
    print('************************')
    print(i)
    print(url)
    category_list=[]
    driver.get(url)
    bs=BeautifulSoup(driver.page_source, "html.parser")
    try:
        dive=bs.find_all('div',class_="ux-layout-section-evo__col")
        for div_ele in dive:
            ander=div_ele.find('span',class_="ux-textspans")
            if ander!=None and ander.text=="Brand":
                brand_finder=div_ele.find('div',class_="ux-labels-values__values")
                df.loc[i, 'brand'] = brand_finder.text
                print(brand_finder.text)
    except:
        print('brand not found')
    try:
        dive=bs.find('div',class_="seo-breadcrumbs-container viexpsvc")
        li=dive.find_all('li')
        for j in range(0,len(li)-1):
            categ=li[j].find('span')
            category_list.append(categ.text)
        df['categories'][i]=category_list
    except:
        df['categories'][i]=[]
    print(category_list)
    
    try:
        dive=bs.find('div',class_="ux-seller-section__item--seller")
        spanner=dive.find('span',class_="ux-textspans ux-textspans--PSEUDOLINK ux-textspans--BOLD")
        seller_info=spanner.text.strip()
        df['seller_info'][i]=seller_info
        print(seller_info)
    except:
        print('no seller found')
        
    try:
        dive=bs.find('div',class_='ebay-content-wrapper')
        rating=dive.find('span',class_='ebay-review-start-rating').text.strip()
        df['average_rating'][i]=rating
        print(rating)
    except:
        print('no rating')

In [ ]:
df.head()

In [ ]:
request_no=input("Enter the request number of the requests seperated with underscore(_) ")
#547_552
file='final_output_ebay_usa_{}.xlsx'.format(request_no)

In [ ]:
df.to_excel(file)

In [ ]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['nitin.bhayana@1digitalstack.ai','ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s mapping request.Please reply as done as you pick this job.'
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()